In [ ]:
import gradio as gr
import requests, base64, io
from PIL import Image

# Endpoint and API key for the Azure ML online endpoint
# For security, prefer storing the key in an environment variable in real projects
# ENDPOINT = os.getenv("AZUREML_ENDPOINT")
# API_KEY  = os.getenv("AZUREML_API_KEY")
ENDPOINT = "<endpoint>"
API_KEY  = "<primary_key>"

def predict(img: Image.Image):
    # Convert PIL image to JPEG bytes then to base64 string
    buf = io.BytesIO()
    img.save(buf, format="JPEG")
    img_b64 = base64.b64encode(buf.getvalue()).decode()

    # Prepare HTTP request to Azure ML endpoint
    headers = {
        "Authorization": f"Bearer {API_KEY}",    # Bearer token auth (endpoint uses key auth)
        "Content-Type": "application/json"
    }
    payload = {"image": img_b64}                 # Payload format expected by scoring.run()

    try:
        # Send POST request and parse JSON response
        r = requests.post(ENDPOINT, json=payload, headers=headers, timeout=30)
        result = r.json()

        # Extract predicted label and probability distribution
        label = result["prediction"]
        probs = result["probabilities"]
        confidence = max(probs) * 100             # Take highest class probability as confidence

        return f"{label} ({confidence:.2f}%)"

    except Exception as e:
        # Return readable error string to the Gradio UI
        return f"Error: {str(e)}"

In [ ]:
# Build and launch a minimal Gradio interface for quick testing
demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Leaf Diseases Classification",
)

# Launch the local Gradio app (set share=True only when needed)
demo.launch(share=True, inline=False)